# 수집한 이미지 데이터 분할 

In [104]:
import os 
os.makedirs('/Users/kibeomkim/Desktop/insta_data/train/samsung')
os.chdir('/Users/kibeomkim/Desktop/insta_data/train')
os.makedirs('/Users/kibeomkim/Desktop/insta_data/train/apple')

os.chdir('/Users/kibeomkim/Desktop/insta_data')

os.makedirs('/Users/kibeomkim/Desktop/insta_data/test/samsung')
os.chdir('/Users/kibeomkim/Desktop/insta_data/test')
os.makedirs('/Users/kibeomkim/Desktop/insta_data/test/apple')
os.chdir('/Users/kibeomkim/Desktop')

In [91]:
os.listdir('/Users/kibeomkim/Desktop/app_sam')

['apple', '.DS_Store', 'samgalaxy']

In [105]:
app_list = [int(x.split('.')[0]) for x in os.listdir('/Users/kibeomkim/Desktop/app_sam/apple') if x != '.DS_Store' ]
sam_list = [int(x.split('.')[0]) for x in os.listdir('/Users/kibeomkim/Desktop/app_sam/samgalaxy') if x != '.DS_Store' ]

In [106]:
import shutil
for image in app_list : 
    if image <= 20 : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/apple/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/test/apple/{image}.jpg')
    else : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/apple/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/train/apple/{image}.jpg')

for image in sam_list : 
    if image <= 20 : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/samgalaxy/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/test/samsung/{image}.jpg')
    else : 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/app_sam/samgalaxy/{image}.jpg', f'/Users/kibeomkim/Desktop/insta_data/train/samsung/{image}.jpg')

In [107]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/insta_data/train/', 
    classes=['samsung', 'apple'],
    target_size=(150, 150),
    batch_size=15, 
    class_mode='binary', 
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/insta_data/test/',
    classes=['samsung', 'apple'],
    target_size=(150,150),
    batch_size=5,
    class_mode='binary', 
)

Found 1778 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [108]:
from keras import models 
from keras import layers 

# VGG16 합성곱 기반 층 만들기 

from keras.applications import vgg16
from keras import optimizers

# 합성곱 기반 층
conv_base = vgg16.VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

# 모델 정의
model = models.Sequential() 
model.add(conv_base)# 합성곱 기반 층
model.add(layers.Flatten()) # 합성곱 기반 층 출력 1차원 텐서(벡터로)변형
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# 합성곱 기반 층 가중치 동결 
conv_base.trainable = False
print(f'{len(model.trainable_weights)}') 


# 모델 컴파일 
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.001),
    metrics=['binary_accuracy']
)

# 모델 훈련 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 118, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=8)

8
Epoch 1/5


2022-03-19 12:47:57.146020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


118/118 [==============================] - ETA: 0s - loss: 0.8701 - binary_accuracy: 0.5837

2022-03-19 12:48:14.806165: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


118/118 [==============================] - 19s 149ms/step - loss: 0.8701 - binary_accuracy: 0.5837 - val_loss: 0.6200 - val_binary_accuracy: 0.6250
Epoch 2/5
118/118 [==============================] - 18s 151ms/step - loss: 0.6235 - binary_accuracy: 0.6784 - val_loss: 0.5438 - val_binary_accuracy: 0.7000
Epoch 3/5
118/118 [==============================] - 17s 142ms/step - loss: 0.6015 - binary_accuracy: 0.6863 - val_loss: 0.5877 - val_binary_accuracy: 0.7250
Epoch 4/5
118/118 [==============================] - 17s 142ms/step - loss: 0.5333 - binary_accuracy: 0.7442 - val_loss: 0.5836 - val_binary_accuracy: 0.6750
Epoch 5/5
118/118 [==============================] - 17s 142ms/step - loss: 0.5077 - binary_accuracy: 0.7510 - val_loss: 0.5755 - val_binary_accuracy: 0.7500


In [96]:
model.save('/Users/kibeomkim/Desktop/75_model.h5') # 모델 저장 

# 미세조정 

In [109]:
conv_base.trainable = True 

set_trainable = False 
for layer in conv_base.layers : 
    if layer.name == 'block5_conv3' : 
        set_trainable = True 
    if set_trainable : 
        layer.trainable = True 
    else : 
        layer.trainable = False 

In [110]:
# 미세조정 

# 컴파일
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.0001),
    metrics=['binary_accuracy']
)
# 훈련
history = model.fit_generator(
    train_generator,
    steps_per_epoch=118, 
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=8,
)

Epoch 1/10


2022-03-19 12:50:02.142419: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


118/118 [==============================] - ETA: 0s - loss: 0.5418 - binary_accuracy: 0.7289

2022-03-19 12:50:21.224561: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


118/118 [==============================] - 20s 155ms/step - loss: 0.5418 - binary_accuracy: 0.7289 - val_loss: 0.6499 - val_binary_accuracy: 0.7500
Epoch 2/10
118/118 [==============================] - 17s 147ms/step - loss: 0.4776 - binary_accuracy: 0.7737 - val_loss: 0.4973 - val_binary_accuracy: 0.7250
Epoch 3/10
118/118 [==============================] - 17s 144ms/step - loss: 0.4262 - binary_accuracy: 0.8009 - val_loss: 0.7498 - val_binary_accuracy: 0.6500
Epoch 4/10
118/118 [==============================] - 17s 146ms/step - loss: 0.3923 - binary_accuracy: 0.8196 - val_loss: 0.6852 - val_binary_accuracy: 0.6750
Epoch 5/10
118/118 [==============================] - 17s 146ms/step - loss: 0.3776 - binary_accuracy: 0.8361 - val_loss: 0.6785 - val_binary_accuracy: 0.6250
Epoch 6/10
 47/118 [==========>...................] - ETA: 10s - loss: 0.3357 - binary_accuracy: 0.8610

KeyboardInterrupt: 

In [102]:
model.save('/Users/kibeomkim/Desktop/80_model.h5') # 모델 저장 

# 일반화 성능 80%, 77%, 75% 세 개 모델 얻었다. 